In [1]:
import nest_asyncio

nest_asyncio.apply()

from crewai import LLM
from dotenv import load_dotenv
import os

load_dotenv()

llm = LLM(
    model=os.getenv("FIREWORKS_MODEL_NAME"),
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY"),
)

In [2]:
from echo.runner import make_call
import asyncio
import nest_asyncio

nest_asyncio.apply()


test_clients = ["Shell", "Schneider electric"]


train_clients = [
    "ICICI bank",
    "Infosys",
    "University of Illinois",
    "Marks and spencer",
    "Mercedes Benz",
]

clients = test_clients + train_clients


inputs = {"call_type": "discovery", "seller": "Whatfix", "n_competitors": 3}

In [3]:
discovery_calls_data = asyncio.run(make_call("discovery", clients, inputs))

Making discovery call for ['Shell', 'Schneider electric', 'ICICI bank', 'Infosys', 'University of Illinois', 'Marks and spencer', 'Mercedes Benz']
Getting analysis Data


Getting Data:   0%|          | 0/7 [00:00<?, ?it/s]

Getting Data for Shell
Adding Analysis Data to Vector Store


Getting Data:  57%|█████▋    | 4/7 [00:02<00:01,  1.97it/s]

Splitting text into documents Discovery Analysis Buyer Data Fields Description: class BuyerDataExtracted(BaseModel):

	pain_points: List[str] # The pain points identified in the call.
	objections: List[str] # The objections identified in the call.
	time_lines: List[str] # The time lines identified in the call.
	success_indicators: List[str] # The success indicators identified in the call.
	budget_constraints: List[str] # The budget constraints identified in the call.
	competition: List[str] # The competitors identified in the call.
	decision_committee: List[str] # The members of the decision committee identified in the call.

# Pain_Points
   - maximizing technology investments
   - driving user adoption
   - balancing oil demand with climate goals
   - not seeing ROI on technology investments
   - users not adopting new tools quickly enough
   - impacting productivity and efficiency
# Objections
   - integration with existing systems
   - customization options
   - cost and implementa

Getting Data: 100%|██████████| 7/7 [00:02<00:00,  2.55it/s]

Getting Data for University of Illinois
Adding Analysis Data to Vector Store
Splitting text into documents Discovery Analysis Buyer Data Fields Description: class BuyerDataExtracted(BaseModel):

	pain_points: List[str] # The pain points identified in the call.
	objections: List[str] # The objections identified in the call.
	time_lines: List[str] # The time lines identified in the call.
	success_indicators: List[str] # The success indicators identified in the call.
	budget_constraints: List[str] # The budget constraints identified in the call.
	competition: List[str] # The competitors identified in the call.
	decision_committee: List[str] # The members of the decision committee identified in the call.

# Pain_Points
   - struggling to provide a seamless onboarding experience for students
   - faculty members are overwhelmed with different systems and applications
   - limited funding for research and programs
   - current onboarding process is not scalable
   - current onboarding proces

In [4]:
# from echo.runner import simulate_calls

# simulation_data = await simulate_calls('discovery', ["CLIENTS NAMES"], inputs, llm)

In [5]:
demo_calls_data = asyncio.run(make_call("demo", clients, inputs))

Making demo call for ['Shell', 'Schneider electric', 'ICICI bank', 'Infosys', 'University of Illinois', 'Marks and spencer', 'Mercedes Benz']
Getting analysis Data


Getting Data:  43%|████▎     | 3/7 [00:00<00:00, 28.47it/s]

Getting Data for Shell
Adding Analysis Data to Vector Store
Splitting text into documents Demo Analysis Buyer Data Fields Description: class BuyerDataExtracted(BaseModel):

	interested_features: List[{
		name: str # The name of the feature of the product or service.
		description: str # A description of the feature.
		pain_point: str # The pain point or challenge that the feature addresses.
	}] # The features of the product or service that stakeholders from the buyer are interested in.
	skeptical_features: List[{
		name: str # The name of the feature of the product or service.
		description: str # A description of the feature.
		objection: str # The objection or concern raised by the buyer.
	}] # The features of the product or service that stakeholders from the buyer are skeptical or have objections, concerns about.
	objections_raised: List[str] # A list of objections raised by the buyer.
	stakeholder_engagement: List[{
		stakeholder: str # The stakeholder's name.
		engagement_level: s

Getting Data: 100%|██████████| 7/7 [00:00<00:00, 28.92it/s]

Node already exists in index: {'seller': 'Whatfix', 'buyer': 'Marks and spencer', 'call_type': 'demo', 'company_size': 'Enterprise', 'industry': 'Retail', 'description': 'M&S is a leading British retailer bringing quality, great value food, clothing and homeware to millions of customers around the world.'}
Getting Data for Mercedes Benz
Adding Analysis Data to Vector Store
Splitting text into documents Demo Analysis Buyer Data Fields Description: class BuyerDataExtracted(BaseModel):

	interested_features: List[{
		name: str # The name of the feature of the product or service.
		description: str # A description of the feature.
		pain_point: str # The pain point or challenge that the feature addresses.
	}] # The features of the product or service that stakeholders from the buyer are interested in.
	skeptical_features: List[{
		name: str # The name of the feature of the product or service.
		description: str # A description of the feature.
		objection: str # The objection or concern raise

In [6]:
pricing_call_data = asyncio.run(make_call("pricing", clients, inputs))

Making pricing call for ['Shell', 'Schneider electric', 'ICICI bank', 'Infosys', 'University of Illinois', 'Marks and spencer', 'Mercedes Benz']
Getting analysis Data


Getting Data:   0%|          | 0/7 [00:00<?, ?it/s]

Getting Data for Shell
Adding Analysis Data to Vector Store
Splitting text into documents Pricing Analysis Buyer Data Fields Description: class BuyerDataExtracted(BaseModel):

	concerns: List[str] # The concerns raised by the buyer during the sales call.
	budget_constraints: List[str] # The budget constraints raised by the buyer during the sales call.
	timelines_to_close: List[str] # The timelines to close mentioned by the buyer during the sales call.
	preferred_pricing_models: List[str] # The preferred pricing models mentioned by the buyer during the sales call.
	KPIs: List[str] # The Key Performance Indicators (KPIs) mentioned by the buyer during the sales call.
	company_financial_priorities: List[str] # The company's financial priorities mentioned by the buyer during the sales call.

# Concerns
   - integration with existing systems
# Budget_Constraints
   - align with budget constraints
# Timelines_To_Close
   - make a decision within the next 2-3 months
# Preferred_Pricing_Models


Getting Data:  43%|████▎     | 3/7 [00:00<00:00, 21.43it/s]

Node already exists in index: {'seller': 'Whatfix', 'buyer': 'ICICI bank', 'call_type': 'pricing', 'company_size': 'Enterprise', 'industry': 'Banking and Financial Services', 'description': 'ICICI Bank is an Indian multinational bank and financial services company headquartered in Mumbai with a registered office in Vadodara.'}
Getting Data for Infosys
Adding Analysis Data to Vector Store
Splitting text into documents Pricing Analysis Buyer Data Fields Description: class BuyerDataExtracted(BaseModel):

	concerns: List[str] # The concerns raised by the buyer during the sales call.
	budget_constraints: List[str] # The budget constraints raised by the buyer during the sales call.
	timelines_to_close: List[str] # The timelines to close mentioned by the buyer during the sales call.
	preferred_pricing_models: List[str] # The preferred pricing models mentioned by the buyer during the sales call.
	KPIs: List[str] # The Key Performance Indicators (KPIs) mentioned by the buyer during the sales ca

Getting Data:  86%|████████▌ | 6/7 [00:00<00:00, 23.63it/s]

Node already exists in index: {'seller': 'Whatfix', 'buyer': 'University of Illinois', 'call_type': 'pricing', 'company_size': '10,001+ employees', 'industry': 'Education', 'description': 'A public research university in the Champaign–Urbana metropolitan area, Illinois, United States.'}
Getting Data for Marks and spencer
Adding Analysis Data to Vector Store
Splitting text into documents Pricing Analysis Buyer Data Fields Description: class BuyerDataExtracted(BaseModel):

	concerns: List[str] # The concerns raised by the buyer during the sales call.
	budget_constraints: List[str] # The budget constraints raised by the buyer during the sales call.
	timelines_to_close: List[str] # The timelines to close mentioned by the buyer during the sales call.
	preferred_pricing_models: List[str] # The preferred pricing models mentioned by the buyer during the sales call.
	KPIs: List[str] # The Key Performance Indicators (KPIs) mentioned by the buyer during the sales call.
	company_financial_prioriti

Getting Data: 100%|██████████| 7/7 [00:00<00:00, 18.40it/s]

Splitting text into documents Pricing Analysis Buyer Data Fields Description: class BuyerDataExtracted(BaseModel):

	concerns: List[str] # The concerns raised by the buyer during the sales call.
	budget_constraints: List[str] # The budget constraints raised by the buyer during the sales call.
	timelines_to_close: List[str] # The timelines to close mentioned by the buyer during the sales call.
	preferred_pricing_models: List[str] # The preferred pricing models mentioned by the buyer during the sales call.
	KPIs: List[str] # The Key Performance Indicators (KPIs) mentioned by the buyer during the sales call.
	company_financial_priorities: List[str] # The company's financial priorities mentioned by the buyer during the sales call.

# Concerns
   - integration with existing systems and tools
   - customization and personalization for users
   - implementation timeline and resource requirements
# Budget_Constraints
   - $10,000 per month
# Timelines_To_Close
   - next 6-12 months
# Preferred

In [7]:
negotiation_call_data = asyncio.run(make_call("negotiation", clients, inputs))

Making negotiation call for ['Shell', 'Schneider electric', 'ICICI bank', 'Infosys', 'University of Illinois', 'Marks and spencer', 'Mercedes Benz']
Getting analysis Data


Getting Data:  57%|█████▋    | 4/7 [00:00<00:00, 30.53it/s]

Getting Data for Shell
Adding Analysis Data to Vector Store
Splitting text into documents Negotiation Analysis Buyer Data Fields Description: class BuyerDataExtracted(BaseModel):

	final_objections: List[str]
	key_decision_makers: List[str]
	remaining_concerns: List[str]
	contract_terms: List[str]

# Final_Objections
   - pricing models
   - integration with existing systems
   - implementation timeline and cost
   - ROI and KPIs
# Key_Decision_Makers
   - Wael Sawan (CEO)
   - Mark (Shell icon)
   - IT
   - Operations
# Remaining_Concerns
   - integration with existing systems
   - implementation timeline and cost
   - balancing oil demand with climate goals
# Contract_Terms
   - custom pricing model
   - implementation timeline of 6-12 weeks

Negotiation Analysis Seller Data Fields Description: class SellerDataExtracted(BaseModel):

	concessions_discounts: List[str]
	objection_handling: List[str]
	stakeholder_consensus: List[str]
	contract_procurement: List[str]
	closing_tactics: Lis

Getting Data: 100%|██████████| 7/7 [00:00<00:00, 27.60it/s]

Node already exists in index: {'seller': 'Whatfix', 'buyer': 'Marks and spencer', 'call_type': 'negotiation', 'company_size': 'Enterprise', 'industry': 'Retail', 'description': 'M&S is a leading British retailer bringing quality, great value food, clothing and homeware to millions of customers around the world.'}
Getting Data for Mercedes Benz
Adding Analysis Data to Vector Store
Splitting text into documents Negotiation Analysis Buyer Data Fields Description: class BuyerDataExtracted(BaseModel):

	final_objections: List[str]
	key_decision_makers: List[str]
	remaining_concerns: List[str]
	contract_terms: List[str]

# Final_Objections
   - pricing
   - implementation timeline
   - resource requirements
# Key_Decision_Makers
   - Mercedes Benz team
# Remaining_Concerns
   - integration with existing systems and tools
   - customization and personalization for users
   - measuring the success of customer experience initiatives
# Contract_Terms
   - custom plan
   - pricing details
   - sc

## Memory Module

## Using Memory

#### Memory Types

- Deal Summary

## Queries

In [8]:
# len(index._storage_context.vector_store._collection.get()['metadatas'])

In [9]:
from echo.queries import get_queries

seller = "Whatfix"
queries = get_queries(seller)

In [10]:
print("Total queries: ", len(queries))

Total queries:  19


In [11]:
from echo.query_executor import arun_queries
from echo.query_executor import ResponseFormat
from echo.query_executor import ContextExtractionMode
import nest_asyncio

nest_asyncio.apply()
import asyncio

inputs = {"buyer": "Shell", "company_size": "Enterprise"}

responses = asyncio.run(
    arun_queries(
        queries,
        inputs=inputs,
        response_format=ResponseFormat.MARKDOWN,
        context_extraction_mode=ContextExtractionMode.QUERY_ENGINE,
    )
)

Running sub query What all pieces of information about the buyer are uncovered in discovery calls of successful deals?
Sub query context {'query': 'What all pieces of information about the buyer are uncovered in discovery calls of successful deals?', 'context': 'Relevant Context:\nThe pain points, objections, time lines, success indicators, budget constraints, competition, and decision committee members are all pieces of information about the buyer that are uncovered in discovery calls of successful deals.'}
Running sub query What all piece of information have we learnt about the buyer from discovery till now?
Sub query context {'query': 'What all piece of information have we learnt about the buyer from discovery till now?', 'context': 'Relevant Context:\nThe buyer, Shell, has identified pain points related to maximizing technology investments, driving user adoption, and balancing oil demand with climate goals. They have objections regarding integration with existing systems, customiza

ValueError: Error interpolating expected_output: Invalid value for key 'query': Unsupported type Query in inputs. Only str, int, float, bool, dict, and list are allowed.

In [4]:
responses["Info To Cover"]

{'response': "# Call Preparation Document\n## Discovery Call with Shell\n### Overview\nThe goal of this discovery call is to uncover the needs and pain points of Shell, a potential buyer, and understand how Whatfix can help them address their challenges.\n\n### Information Uncovered so far\n#### Challenges and Pain Points\n* Maximize technology investments and driving user adoption\n* Balancing oil demand with climate goals\n* Slow user adoption of new tools, impacting productivity and efficiency\n* Integration with existing systems\n* High degree of customization to meet unique requirements\n* Concerns about cost and implementation timeline of a digital adoption platform\n\n#### Decision Committee Members\n* IT\n* Operations\n* Business Leaders\n* CEO Wael Sawan\n* Mark (Shell icon)\n* Shareholders\n\n### Missing Information to Uncover\n#### From Historical Data\n* Objections\n* Time lines\n* Success indicators\n* Budget constraints\n* Competition\n\n#### From Current Call\n* None, as

In [1]:
# from echo.query_executor import aget_query_response


# response, sub_queries_context = await aget_query_response(
#     query,
#     inputs,
#     response_format=ResponseFormat.MARKDOWN, context_extraction_mode=ContextExtractionMode.QUERY_ENGINE
# )

In [ ]:
from echo.step_templates.discovery import aget_research_data_for_client
from echo.utils import get_llm
import nest_asyncio

nest_asyncio.apply()
import asyncio


data = asyncio.run(aget_research_data_for_client(inputs, llm=get_llm()))